In [1]:
import pandas as pd
import numpy as np

In [2]:
# title.basics.tsv.gz url
basics_url="https://datasets.imdbws.com/title.basics.tsv.gz"
# load basic URL
basics = pd.read_csv(basics_url, sep='\t', low_memory = False)

KeyboardInterrupt: 

In [ ]:
basics.head()

In [ ]:
# title.ratings.tsv.gz
ratings_url = "https://datasets.imdbws.com/title.ratings.tsv.gz"
# Load raitings URL
ratings = pd.read_csv(ratings_url, sep='\t', low_memory = False)

In [ ]:
ratings.head()

In [ ]:
# title.akas.tsv.gz
akas_url = "https://datasets.imdbws.com/title.akas.tsv.gz"
# Load akas URL
akas = pd.read_csv(akas_url, sep='\t', low_memory = False)

In [ ]:
akas.head()

## Replacing \N with nan

In [ ]:
# Replacing \N in basics
basics.replace({"\\N":np.nan}, inplace = True)

In [ ]:
# Confirm Replacement 
basics.head()

In [ ]:
# Replace \N in ratings
ratings.replace({"\\N":np.nan}, inplace = True)

In [ ]:
#Replace \N in akas
akas.replace({"\\N":np.nan}, inplace = True)

In [ ]:
# Confirm replacement 
akas.head()

## Basics Processing

In [ ]:
basics.head()

In [ ]:
# Eliminate movies that are null for runtimeMinutes 
basics.dropna(subset = ["runtimeMinutes"], inplace = True)

In [ ]:
# Confirm drop
basics["runtimeMinutes"].isnull().sum()

In [ ]:
# Eliminate movies that are null for genre
basics.dropna(subset = ["genres"], inplace = True)

In [ ]:
# Confirm Drop 
basics["genres"].isnull().sum()

In [ ]:
basics["titleType"].value_counts()

In [ ]:
# keep only titleType==Movie
basics = basics.loc[(basics["titleType"]=="movie")]

In [ ]:
# Confirm 
basics.head()

In [ ]:
# keep startYear 2000-2022
basics = basics.loc[(basics["startYear"]>"1999")&(basics["startYear"]<"2022")]

In [ ]:
# Confirm
basics.head()

In [ ]:
# Eliminate movies that include  "Documentary" in genre (see tip below)
is_doc = basics["genres"].str.contains("documentary", case = False)
basics = basics[~is_doc]

In [ ]:
#Confirm
basics.head()

## AKAS Processing

In [ ]:
akas.head()

In [ ]:
# keep only US entries
akas = akas.loc[(akas["region"]== "US")]

In [ ]:
# Confirm
akas.head()

## Filtering dataframes

In [ ]:
# Filter the basics table down to only include the US by using the filter akas dataframe
keepers = basics["tconst"].isin(akas["titleId"])
keepers

In [ ]:
# filter basics
basics = basics[keepers]
basics

In [ ]:
basics.info()

In [ ]:
ratings.info()

In [ ]:
akas.info()

In [ ]:
# # example making new folder with os
import os
os.makedirs('Data/',exist_ok=True) 
# Confirm folder created
os.listdir("Data/")

In [ ]:
# Save basics
basics.to_csv("Data/title_basics.csv.gz",compression='gzip',index=False)

In [ ]:
# Confirm
basics = pd.read_csv("Data/title_basics.csv.gz", low_memory = False)
basics.head()

In [ ]:
# Save ratings
ratings.to_csv("Data/title_ratings.csv.gz", compression = "gzip", index = False)

In [ ]:
# confirm 
ratings = pd.read_csv("Data/title_ratings.csv.gz", low_memory = False)
ratings.head()

In [ ]:
# Save akas 
akas.to_csv("Data/title_akas.csv.gz", compression = "gzip", index = False)

In [ ]:
# confrim 
akas = pd.read_csv("Data/title_akas.csv.gz", low_memory = False)
akas.head()

## Project 3 - Part 2

In [ ]:
# Install tmbdsimple 
!pip install tmdbsimple

In [ ]:
import json
with open("/Users/samlagana/.secret/tmdb_api.json", "r") as f:
    login = json.load(f)
# Display keys of dict
login.keys()

In [ ]:
import tmdbsimple as tmdb
tmdb.API_KEY = login["api-key"]

In [ ]:
# Make a movie object using the .Movies function from tmdb
movie = tmdb.Movies(603)

# Movie objects .info dict
info = movie.info()
info

In [ ]:
# Movie with rating function 
def get_movie_with_rating(movie_id):
    # Get the movie object for current id
    movie = tmdb.Movies(movie_id)
    # save the .info .releases dict
    movie_info = movie.info()
    releases = movie.releases()
    # Loop through countries in releases
    for c in releases["countries"]:
        # if the country is == US
        if c["iso_3166_1"]== "US":
            movie_info["certification"]= c["certification"]
    return movie_info

In [ ]:
get_movie_with_rating("603")

In [ ]:
import os 
from tqdm.notebook import tqdm_notebook
FOLDER = "Data/"
os.makedirs(FOLDER, exist_ok = True)
os.listdir(FOLDER)

In [ ]:
def write_json(new_data, filename):
    """"Adapted from: https://www.geeksforgeeks.org/append-to-json-file-using-python/"""
    
    with open(filename, "r+") as file:
        # First we load existing data into dict
        file_data = json.load(file)
        #chose extend or append
        if (type(new_data)==list) & (type(file_data)== list):
            file_data.extend(new_data)
        else:
            file_data.append(new_data)
        # Sets file's current position as offset
        file.seek(0)
        # convert back to json 
        json.dump(file_data, file)

In [ ]:
# Load df from project part 1 as basics
basics = pd.read_csv("Data/title_basics.csv.gz")
basics.head()

In [ ]:
YEARS_TO_GET = [2000,2001]

In [ ]:
YEAR = YEARS_TO_GET[0]
YEAR

In [ ]:
# Start of outer loop 
for YEAR in tqdm_notebook(YEARS_TO_GET, desc = "YEARS",
                         position = 0):

    # Defining JSON file to store results for year
    JSON_FILE = f"{FOLDER}tmdb_api_results_{YEAR}.json"
    # Check if file exists
    file_exists = os.path.isfile(JSON_FILE)


    # if it does not exist
    if file_exists == False:
        # save an empty dict with just "imdb_id" to the new json file
        with open(JSON_FILE,"w") as f:
            json.dump([{"imdb_id":0}],f)

    #Saving new year as the current df
    df = basics.loc[basics["startYear"]==YEAR].copy()
    #saving movide ids to list
    movie_ids= df["tconst"].copy()#.to_list()
    movie_ids

    # Load in existing data from json info df called "previous_df"
    previous_df = pd.read_json(JSON_FILE)
    previous_df

    # filter out any ids that are already in the JSON FILE
    movie_ids_to_get = movie_ids[~movie_ids.isin(previous_df["imdb_id"])]

    # Get index and movie id from list
    # INNER Loop 
    for movie_id in tqdm_notebook(movie_ids_to_get,
                                 desc= f"Movies from{YEAR}",
                                 position = 1,
                                 leave = True):
        # Attempt to retrieve the data for the movie id 
        try: 
            temp = get_movie_with_rating(movie_id) # uses pre-made function
                #Append/Extend results to exisiting file using premade funct
            write_json(temp,JSON_FILE)
            # Short 20 ms sleep to prevent overwhelming server
            time.sleep(0.02)
        # if it fails, make a dict with just id and none for certification
        except Exception as e:
            continue
   
    # Saving final df
    final_year_df = pd.read_json(JSON_FILE)
    final_year_df.to_csv(f"{FOLDER}final_tmdb_data{YEAR}.csv.gz",compression = "gzip", index = False)